<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Прецессия Меркурия

### Examples - Astrophysics
<section class="post-meta">
By Eilif Sommer Øyre, Jonas Tjemsland and Jon Andreas Støvneng
</section>
Last edited: April 28th 2018 
___

В этом блокноте мы обсудим общую релятивистскую прецессию орбиты Меркурия вокруг Солнца. Мы будем использовать понятия и формулы, введенные и выведенные в блокноте по [Общей теории относительности](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/GR.ipynb).

Перигелий орбиты Меркурия увеличивается на 5300 угловых секунд в столетие, но только 5260 угловых секунд могут быть объяснены ньютоновскими эффектами [1]. Наша цель здесь - проверить, может ли общая теория относительности объяснить последние $\sim 40$ угловых секунд.

Обратите внимание, что поправка довольно мала, и поэтому мы не можем наивно продолжать вычислять орбиту Меркурия в течение одного столетия (или одного периода) и проверять прецессию. В этом блокноте мы вычислим прецессию для набора орбит с более высокой скоростью прецессии, выполним линейную регрессию и экстраполируем прецессию Меркурия. Когда выражение мало, скорость прецессии может быть аппроксимирована аналитическим выражением. Это будет обсуждаться в конце блокнота.

Мы начинаем с импорта пакетов, установки параметров фигуры и определения некоторых физических констант.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from scipy.optimize import curve_fit
import time
%matplotlib inline

In [ ]:
# Set common figure parameters
newparams = {'figure.figsize': (16, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 14}
plt.rcParams.update(newparams)

# Some constants
AU = 149597870700  # m. Astronomical Unit. Avg. distance Earth Sun
T = 365.25*3600*24 # s. 1 year
c = 299792458      # AU/yr. Speed of light
G = 6.67408e-11    # m^3/kg s^2. Gravitational constant
M = 1.989e30       # kg. Solar mass

### Безразмерное уравнение движения

Можно показать (см. Наш блокнот по [Общей теории относительности](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/GR.ipynb) или ref. [2]) что сила на планете массой $m\ll M$ вне сферически симметричного распределения массы массы $M$ задается

\begin{equation}
\mathbf F =-\frac{GMm}{r^3}\mathbf r\left(1 + \frac{3l^2}{c^2r^2}\right),
\label{eq:force} \quad (1)
\end{equation}

где $G$ - ньютоновская гравитационная постоянная, $\vec r = (x, y, z)$ - расстояние до центра распределения массы, $l$ - угловой момент на единицу массы и $c$ - скорость света. Орбита планеты будет ограничена одной плоскостью из-за сохранения углового момента. Таким образом, мы можем ограничить обсуждение плоскостью $xy$.

Пусть $R=1\,\mathrm{AU}$ - расстояние между Землей и Солнцем, $T=1\,\mathrm{год}$, $m$ - масса Меркурия и $M$ - масса Солнца. Определите $t_0\equiv t/T$, $\rho\equiv r/R$, $X\equiv x/R$ и $Y\equiv y/R$. В этом случае мы можем записать уравнение (1) в безразмерной форме

\begin{equation}
\frac{\mathrm{d}^2}{\mathrm{d}t_0^2}\left(X, Y\right) =-\frac{GMT^2}{R^3}\frac{\left(X, Y\right)}{\rho^3}\left(1+\frac{3l^2}{R^2c^2\rho^2}\right)\equiv -A\frac{\left(X, Y\right)}{\rho^3}\left(1+\frac{B}{\rho^2}\right).
\label{eq:dimless} \quad (2)
\end{equation}

### Нахождение $A$ и $B$

Используя законы Кеплера (см., например, [1]), легко показать, что

$$M_\odot =\frac{4\pi^2\cdot R^3}{G\cdot T^2}.$$

Константа $A$, таким образом,
$$A = 4\pi^2.$$

In [ ]:
A = 4*np.pi**2

Чтобы найти $B$, мы должны сначала найти значение углового момента (на единицу массы) $l$. При этом нам понадобятся следующие данные [3]:

| Mercury                                                | |
| ---------------------------------------- |:-------------:|
| Perihelion (closest distance to the Sun) | 0.307 499 AU  | 
| Aphelion (farthest distance to the Sun)  | 0.466 697 AU  |
| Orbital period                           | 0.240 846 yr  |
| Max. orbital speed                       | 58.98 km/s    |
| Min. orbital speed                       | 38.86 km/s    |

В перигелии скорость максимальна. В афелии она минимальна. В обоих случаях скорость нормальна к вектору расстояния между Солнцем и Меркурием. Таким образом, угловой момент на единицу массы $l=|\vec v\times \vec r|$ задается

$$l = v_\mathrm{max}\cdot r_\mathrm{min}=v_\mathrm{min}\cdot r_\mathrm{max}.$$

In [ ]:
perihelion = 0.307499*AU # m
aphelion = 0.466697*AU   # m
maxVel = 58.98*1000      # m/s
minVel = 38.86*1000      # m/s
l1 = aphelion*minVel
l2 = perihelion*maxVel
print("l = %.5e c^2 = %.5e c^2"%(l1/c**2, l2/c**2))
l = (l1 + l2)/2

Давайте вычислим константу $B$.

In [ ]:
B = 3*l**2/(c*AU)**2
print("B = %.3e"%(B))

Это небольшое число, что означает, что общая релятивистская поправка на ньютоновскую орбиту невелика. Поскольку поправка настолько мала, проблему можно представить как возмущение и "решить" аналитически (см. Обсуждение в конце урока).

На самом деле, если мы продолжим наивно вычислять орбиту Меркурия, используя уравнение (2), мы получим неправильную скорость прецессии из-за ошибок численного округления. Поэтому мы рассмотрим различные значения для $B$, выполним регрессию и извлечем скорость прецессии для правильного $B$.

### Численная Реализация

Теперь мы реализуем тот же код, что и в [Общей теории относительности](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/GR.ipynb). Поскольку вычисления требуют больших затрат, мы написали подпрограмму Fortran, которая вычисляла орбиту Меркурия. Код приведен в приложении ниже. Более конкретно, код принимает начальную позицию $(X, Y)$, начальную скорость $(U, V)$, длину шага $h$ и константы $A$ и $B$ в качестве входных данных и возвращает позицию в перигелии. То есть он вычисляет траекторию до тех пор, пока радиус не достигнет (локального) максимума.

Код компилируется с помощью F2PY (см. Нашу [страницу](https://www.numfys.net/howto/F2PY/) для общего введения)

```
f2py -c filename.f95 -m mercury
```
Модуль расширения затем называется "меркурий" и может быть импортирован следующим образом.

In [ ]:
import mercury
print(mercury.__doc__)

Если мы позволим орбите начинаться с афелия, прецессия $\phi$ после периода $1/2$ может быть найдена с помощью простой тригонометрии. Как и в предыдущем условии, мы позволяем Меркурию находиться в перигелии (минимальное расстояние) при $(x, y) = (0, -r_\mathrm{min})$. Пусть $(x', y')$ - позиция в первом афелии (максимум). Для простоты и для удобства чтения кода мы предполагаем, что прецессия в каждом рассматриваемом случае достаточно мала. Прецессия в этом случае может быть аппроксимирована

\begin{equation}
\phi\approx x/y.
\end{equation}

**Упражнение:** Хорошее ли это приближение?

In [ ]:
Z0 = [0, -perihelion/AU, maxVel*T/AU, 0] # Initial condition

def getPrecession(X, Y):
    """ Return the precession (half period) in radians. """
    radians = -np.arctan(X/Y)
    return radians

Нам также нужно преобразовать радианы за полупериод в дуговые секунды в столетие. Одна угловая секунда определяется как $1/3600$ часть градуса. Орбитальный период Меркурия составляет 0,240 года (см. таблицу выше).

In [ ]:
def r2aspc(radians):
    """ Converts radians to arcseconds per century for Mercury. """
    return 100*2*radians*(180/np.pi)*3600/0.240846

Нам нужно выбрать длину шага $h$, но насколько он мал? Одним из способов проверки точности кода является проверка того, что прецессия, полученная для $B=0$, на несколько порядков меньше, чем прецессия, полученная для $B>0$.

Мы вычислим прецессию для набора $B$-констант между $10^{-3}$ и $10^{-4}$ и выполним линейную регрессию, как объяснялось ранее.

In [ ]:
B0 = np.logspace(-3, -4, 20)
Tmax = 0.28  # yr
n = 1000000  # step
h = n/Tmax

In [ ]:
# Check that n is large enough
X, Y, steps = mercury.getorbit(Z0, Tmax/float(n), A, 0)
p0 = getPrecession(X, Y)
X, Y, steps = mercury.getorbit(Z0, Tmax/float(n), A, B0[-1])
p1 = getPrecession(X, Y)
print("With precession:    %.1e"%(abs(p1)))
print("Without precession: %.1e"%(abs(p0)))

Теперь мы можем выполнить линейную регрессию и экстраполировать прецессию меркурия.

In [ ]:
phi = np.zeros(len(B0))
for i in range(len(B0)):
    X, Y, steps = mercury.getorbit(Z0, Tmax/float(n), A, B0[i])
    phi[i] = getPrecession(X, Y)

Когда $B=0$, прецессия должна быть равна нулю. Таким образом, мы помещаем функцию в форму $f(x)=c\times x$, используя функцию `curve_fit` из `scipy.optimize`.

In [ ]:
def f(x, a):
    return a*x

popt, pcov = curve_fit(f, B0, phi)

In [ ]:
plt.figure()
plt.plot(B0, B0*popt[0], label="Fit")
plt.plot(B0, phi, "o", label="Data points")
plt.xlabel(r"GR correction constant, $B$")
plt.ylabel(r"Precession, $\phi$ [radians per half orbit]")
plt.legend()
plt.show()

In [ ]:
print("Precession of Mercury is estimated to %.2f arcsec per century."
      % (r2aspc(B*popt[0])))

Можно показать, что скорость прецессии задается [2]

\begin{equation}
\delta \phi_\mathrm{prec}=\frac{6\pi G}{c^2}\frac{M}{a\left(1-\epsilon^2\right)}.
\end{equation}

Эксцентриситет орбиты равен $\epsilon=0.2056$ и полу-главная ось равна $57.91\times 10^6\;\mathrm{km}$ [3].

In [ ]:
epsilon = 0.2056
a = 57.91e9
dphi = 6*np.pi*G*M/(c**2*a*(1 - epsilon**2))
print("Precession of Mercury is approximately %.2f arcsec per century."
      % (dphi*100*3600/0.240846*180/np.pi))

Ожидаемый результат - $43$ arcsec. Результаты отображаются в пределах двух цифр. Как и ожидалось: оба метода включают несколько приближений, и в численном случае существуют некоторые ошибки численного округления.

# Resources
[1] Kutner, M. (2003). Astronomy: A Physical Perspective. Cambridge: Cambridge University Press. doi:10.1017/CBO9780511802195  
[2] Hartle, J. B. Gravity: An Introduction to Einstein's General Relativity. San Francisco: Addison-Wesley, 2003.   
[3] NASA. Mercury Fact Sheet. URL: https://nssdc.gsfc.nasa.gov/planetary/factsheet/mercuryfact.html (accessed: Feb. 2018)

# Appendix: Fortran (95) code

```fortran

subroutine getOrbit(x, y, steps, z0, h, a, b)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!! This code is used to compute the position of a planets position  !!
!! at aphelion (maximum) in the Schwarzschild geometry given        !!
!! initial positions and velocities. The code is described in       !!
!! detail (Python) in the General relativity notebook on            !!
!! numfys.net. The code can be compiled using F2PY.                 !!
!!                                                                  !!
!! Input:                                                           !!
!!    z0 : float, len(4). [x, y, u, v]. Initial x and y position    !!
!!         and initial velocity in x and y direction.               !!
!!    h  : float. Step length.                                      !!
!!    a  : float. Constant A (force strength).                      !!
!!    b  : float. Constant B (GR correction).                       !!
!! Output:                                                          !!
!!    x  : float. x position at next maximum.                       !!
!!    y  : float. y position at next maximum.                       !!
!!    steps: int. Number of steps used.                             !!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

implicit none

integer, intent(out)  :: steps
double precision, intent(out) :: x, y
double precision, intent(in) :: z0(4), a, b, h
double precision :: z(4), rho2, rhotemp

z = z0

rhotemp = 0
rho2 = z0(1)**2 + z0(2)**2
steps = 0

! Uses 4th order Runge-Kutta method to compute the position of the
! planet at each step. The mass distribution (Sun) is located around
! the origin. When the distance to the origin do not increase, we
! are at the maximum (aphelion).
do while (rho2 > rhotemp)
    rhotemp = rho2
    call rk4step(a, b, z, h)
    rho2 = z(1)**2 + z(2)**2
    steps = steps + 1
end do

x = z(1)
y = z(2)

return

contains

    ! Right hand side of EoM
    subroutine rhs(a, b, z, s)

        implicit none

        double precision, intent(in) :: z(4)
        double precision, intent(in)    :: a, b
        double precision, intent(out) :: s(4)
        double precision :: temp, rho

        rho = (z(1)**2 + z(2)**2)**.5
        temp = a/rho**3*(1 + B/rho**2)

        s(1) = z(3)
        s(2) = z(4)
        s(3) = - z(1)*temp
        s(4) = - z(2)*temp

        return

    end subroutine

    ! One step of Fourth-Order Runge-Kutta Method
    subroutine rk4step(a, b, z, h)

        implicit none

        double precision, intent(inout) :: z(4)
        double precision, intent(in) :: a, b, h
        double precision :: s1(4), s2(4), s3(4), s4(4)

        call rhs(a, b, z, s1)
        call rhs(a, b, z + h*s1/2.0, s2)
        call rhs(a, b, z + h*s2/2.0, s3)
        call rhs(a, b, z + h*s3, s4)

        z = z + h/6.0*(s1 + 2.0*s2 + 2.0*s3 + s4)

        return

    end subroutine

end subroutine

```